<a href="https://colab.research.google.com/github/Barytes/NBS_project/blob/Plot/plot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys

def is_colab():
    return 'google.colab' in sys.modules

if is_colab():
    from google.colab import drive
    drive.mount('/content/drive')
    PROJECT_PATH = '/content/drive/MyDrive/NBS_project/'
else:
    # 自动根据当前脚本定位项目根路径
    PROJECT_PATH = os.path.abspath(os.path.join(os.path.dirname(__file__), '..')) \
        if '__file__' in globals() else os.getcwd()

if PROJECT_PATH not in sys.path:
    sys.path.append(PROJECT_PATH)

print(f"项目路径设置为: {PROJECT_PATH}")

from src.config import Config
from src.main import create_MDs,create_ESP
import src.baseline as baseline
import src.NBS as NBS
import src.model as model
import numpy as np
import matplotlib.pyplot as plt


Mounted at /content/drive
项目路径设置为: /content/drive/MyDrive/NBS_project/


In [ ]:
CONFIG_PATH = os.path.join(PROJECT_PATH, "configs", "test.json")
config = Config(CONFIG_PATH)

In [ ]:
# Experiment 1: Performance of Proposed Algorithm

num_mds = [10, 20, 30, 40, 50]
sw_uni, sw_pro, sw_non, sw_con, sw_swm, sw_admm = [],[],[],[],[],[]
Dm_uni, Dm_pro, Dm_non, Dm_con, Dm_swm, Dm_admm = [],[],[],[],[],[]
baselines = ['Uni', 'Pro', 'Non', 'Con', 'Opt', 'ADMM']
for n in num_mds:
  config.params["num_mds"] = n
  print(f"Number of MDs: {n}")

  seed = 41
  MDs = create_MDs(config, seed=seed)
  esp = create_ESP(config, seed=seed)
  # np.random.shuffle(MDs)  # Shuffle the MDs
  # for md in MDs[:n]:
  #   print(md.param)
  # print(esp.param)

  T = 100
  N = len(MDs)
  lam_uni, p_uni, r_uni, Dmax_uni = np.zeros(N),np.zeros(N),np.zeros(N),0
  for _ in range(T):
      l,p,r,d = baseline.uniform_baseline(esp, MDs,43)
      lam_uni += l
      p_uni += p
      r_uni += r
      Dmax_uni += d
  lam_uni, p_uni, r_uni, Dmax_uni = lam_uni/T, p_uni/T, r_uni/T, Dmax_uni/T

  lam_pro, p_pro, r_pro, Dmax_pro = np.zeros(N),np.zeros(N),np.zeros(N),0
  for _ in range(T):
      l,p,r,d = baseline.proportional_baseline(esp,MDs,43)
      lam_pro += l
      p_pro += p
      r_pro += r
      Dmax_pro += d
  lam_pro, p_pro, r_pro, Dmax_pro = lam_pro/T, p_pro/T, r_pro/T, Dmax_pro/T

  lam_non, p_non, r_non, Dmax_non = baseline.non_cooperative_baseline(esp,MDs)

  lam_con, p_con, r_con, Dmax_con = baseline.contract_baseline(esp,MDs)

  lam_swm, p_swm, Dmax_swm = baseline.social_welfare_maximization(esp, MDs)

  lamb_admm, p_admm, Dmax_admm = NBS.ADMM(esp, MDs)
  r_admm = NBS.negotiation(esp, MDs,lamb_admm, p_admm, Dmax_admm)

  sw_uni.append(model.social_welfare(esp, MDs, lam_uni, p_uni))
  sw_pro.append(model.social_welfare(esp, MDs, lam_pro, p_pro))
  sw_non.append(model.social_welfare(esp, MDs, lam_non, p_non))
  sw_con.append(model.social_welfare(esp, MDs, lam_con, p_con))
  sw_swm.append(model.social_welfare(esp, MDs, lam_swm, p_swm))
  sw_admm.append(model.social_welfare(esp, MDs, lamb_admm, p_admm))
  Dm_uni.append(Dmax_uni)
  Dm_pro.append(Dmax_pro)
  Dm_non.append(Dmax_non)
  Dm_con.append(Dmax_con)
  Dm_swm.append(Dmax_swm)
  Dm_admm.append(Dmax_admm)
  print("===================")
sw = [sw_uni, sw_pro, sw_non, sw_con, sw_swm, sw_admm]
Dm = [Dm_uni, Dm_pro, Dm_non, Dm_con, Dm_swm, Dm_admm]
plt.figure(figsize=(10, 6)) # 设置图的大小
for i, data in enumerate(sw):
    plt.plot(N, data, marker='o', label=f'SW_{baselines[i]}')

plt.title('Social Welfare vs. N')
plt.xlabel('N')
plt.ylabel('Social Welfare')
plt.xticks(N) # 确保 N 值在横轴上清晰显示
plt.grid(True)
plt.legend()
plt.show()

plt.figure(figsize=(10, 6)) # 设置图的大小
for i, data in enumerate(Dm):
    plt.plot(N, data, marker='o', label=f'Delay_{baselines[i]}')

plt.title('Delay vs. N')
plt.xlabel('N')
plt.ylabel('Delay')
plt.xticks(N) # 确保 N 值在横轴上清晰显示
plt.grid(True)
plt.legend()
plt.show()

Number of MDs: 10

--- Outer iter 0: mu=0.01, rho=100 ---


/content/drive/MyDrive/NBS_project/src/baseline.py:69: RuntimeWarning: divide by zero encountered in scalar power
  return pi - 2*c[i]*p - k[i]*(F[i]-p)**(k[i]-1)


Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.2060322298191153
            Iterations: 191
            Function evaluations: 8272
            Gradient evaluations: 188
  inner NLP status = Optimization terminated successfully

--- Outer iter 1: mu=0.008, rho=130 ---
Optimization terminated successfully    (Exit mode 0)
            Current function value: 23.434159667244415
            Iterations: 31
            Function evaluations: 1189
            Gradient evaluations: 27
  inner NLP status = Optimization terminated successfully

--- Outer iter 2: mu=0.0064, rho=169 ---
Optimization terminated successfully    (Exit mode 0)
            Current function value: 53.711973963312246
            Iterations: 21
            Function evaluations: 891
            Gradient evaluations: 20
  inner NLP status = Optimization terminated successfully

--- Outer iter 3: mu=0.00512, rho=219.7 ---
Positive directional derivative for linesearch    (Exit mode

/usr/local/lib/python3.11/dist-packages/scipy/optimize/_differentiable_functions.py:552: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(delta_x, delta_g)
/usr/local/lib/python3.11/dist-packages/scipy/optimize/_differentiable_functions.py:317: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)


✅ 这个点严格满足所有约束（在容差范围内）。
Optimization terminated successfully    (Exit mode 0)
            Current function value: -87.69425904146827
            Iterations: 43
            Function evaluations: 962
            Gradient evaluations: 43


/usr/local/lib/python3.11/dist-packages/scipy/optimize/_slsqp_py.py:435: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)


Number of MDs: 20

--- Outer iter 0: mu=0.01, rho=100 ---
Optimization terminated successfully    (Exit mode 0)
            Current function value: -51.100203636219696
            Iterations: 78
            Function evaluations: 6471
            Gradient evaluations: 77
  inner NLP status = Optimization terminated successfully

--- Outer iter 1: mu=0.008, rho=130 ---
Optimization terminated successfully    (Exit mode 0)
            Current function value: -41.98206313575102
            Iterations: 60
            Function evaluations: 4756
            Gradient evaluations: 56
  inner NLP status = Optimization terminated successfully

--- Outer iter 2: mu=0.0064, rho=169 ---
Optimization terminated successfully    (Exit mode 0)
            Current function value: -30.453602909379057
            Iterations: 49
            Function evaluations: 3814
            Gradient evaluations: 45
  inner NLP status = Optimization terminated successfully

--- Outer iter 3: mu=0.00512, rho=219.7 ---
Op

/content/drive/MyDrive/NBS_project/src/baseline.py:205: RuntimeWarning: invalid value encountered in scalar power
  return np.asarray([md.cn*(pn**2)+(md.Fn)**md.kn-(md.Fn-pn)**md.kn for md, pn in zip(MDs, p)])


ValueError: array must not contain infs or NaNs

In [ ]:
# Experiment 2: Comparison of NBS and Non-Cooperative

In [ ]:
# Experiment 3: Impact of Bargaining Power

In [ ]:
# Experiment 4: Impact of Task Workload

In [ ]:
# Experiment 5: Impact of QoS Requirement